1.Посчитайте, сколько компаний закрылось.

In [ ]:
query = '''
SELECT COUNT(status)
FROM company
WHERE status = 'closed'

'''
pd.io.sql.read_sql(query, con = engine)

2.Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
query = '''
SELECT SUM(funding_total) AS ft
FROM company
WHERE country_code = 'USA' AND category_code = 'news'
GROUP BY name
ORDER BY ft DESC
'''
pd.io.sql.read_sql(query, con = engine)

3.Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
query = '''
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
AND CAST(acquired_at AS date) BETWEEN '2011-01-01' AND '2013-12-31'
'''
pd.io.sql.read_sql(query, con = engine)

4.Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
query = '''
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'
'''
pd.io.sql.read_sql(query, con = engine)

5.Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
query = '''
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%'
'''
pd.io.sql.read_sql(query, con = engine)

6.Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
query = '''
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC
'''
pd.io.sql.read_sql(query, con = engine)

7.Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
query = '''
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0
AND MIN(raised_amount) != MAX(raised_amount)
'''
pd.io.sql.read_sql(query, con = engine)

8.Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
query = '''
SELECT *,
       CASE
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
        ELSE 'low_activity'
        END
FROM fund
'''
pd.io.sql.read_sql(query, con = engine)

9.Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
query = '''
SELECT ROUND(AVG(investment_rounds)) AS round,
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund
GROUP BY activity
ORDER BY round
'''
pd.io.sql.read_sql(query, con = engine)

10.Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.
Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
query = '''
SELECT country_code AS country,
       MAX(invested_companies) AS max_inv,
       MIN(invested_companies) AS min_inv,
       AVG(invested_companies) AS avg_inv
FROM FUND
WHERE EXTRACT (YEAR FROM CAST(founded_at AS timestamp)) BETWEEN '2010' AND '2012'
GROUP BY country_code
HAVING MIN(invested_companies) != 0
ORDER BY avg_inv DESC
LIMIT 10
'''
pd.io.sql.read_sql(query, con = engine)

11.Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
query = '''
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p LEFT JOIN education AS e ON p.id = e.person_id
'''
pd.io.sql.read_sql(query, con = engine)

12.Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
query = '''
SELECT name,
       COUNT(DISTINCT(e.instituition))
FROM people AS p JOIN education AS e ON p.id = e.person_id
JOIN company AS c ON c.id = p.company_id
GROUP BY name
ORDER BY count DESC
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

13.Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
query = '''
SELECT DISTINCT(c.name)
FROM company AS c LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE is_first_round = 1 AND is_last_round = 1 AND status = 'closed'
'''
pd.io.sql.read_sql(query, con = engine)

14.Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
query = '''
SELECT id
FROM people
WHERE company_id in (SELECT DISTINCT(c.id)
FROM company AS c LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE is_first_round = 1 AND is_last_round = 1 AND status = 'closed')
'''
pd.io.sql.read_sql(query, con = engine)

15.Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
query = '''
SELECT DISTINCT(p.id),
       e.instituition
FROM people AS p LEFT JOIN education AS e ON p.id = e.person_id
WHERE person_id in
(SELECT id AS people_id
FROM people
WHERE company_id in (SELECT DISTINCT(c.id)
FROM company AS c LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE is_first_round = 1 AND is_last_round = 1 AND status = 'closed'))
'''
pd.io.sql.read_sql(query, con = engine)

16.Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
query = '''
SELECT id,
       COUNT(instituition) AS count_inst
       
FROM 
(SELECT (p.id),
       e.instituition
FROM people AS p LEFT JOIN education AS e ON p.id = e.person_id
WHERE person_id in
(SELECT id AS people_id
FROM people
WHERE company_id in (SELECT (c.id)
FROM company AS c LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE is_first_round = 1 AND is_last_round = 1 AND status = 'closed'))) AS i
GROUP BY id
'''
pd.io.sql.read_sql(query, con = engine)

17.Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
query = '''
SELECT AVG(count_inst)
FROM
(SELECT id,
       COUNT(instituition) AS count_inst
       
FROM 
(SELECT (p.id),
       e.instituition
FROM people AS p LEFT JOIN education AS e ON p.id = e.person_id
WHERE person_id in
(SELECT id AS people_id
FROM people
WHERE company_id in (SELECT (c.id)
FROM company AS c LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE is_first_round = 1 AND is_last_round = 1 AND status = 'closed'))) AS i
GROUP BY id) AS i
'''
pd.io.sql.read_sql(query, con = engine)

18.
Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
query = '''
SELECT AVG(total)
FROM (SELECT DISTINCT p.id, COUNT(e.instituition) AS total
FROM people AS p
JOIN education AS e ON p.id = e.person_id
WHERE company_id IN (SELECT DISTINCT id
FROM company 
WHERE name in ('Facebook')) 
GROUP BY p.id) AS t1
'''
pd.io.sql.read_sql(query, con = engine)

19.Составьте таблицу из полей:
* name_of_fund — название фонда;
* name_of_company — название компании;
* amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
query = '''
SELECT f.name AS name_of_found,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i LEFT JOIN company AS c on i.company_id=c.id
LEFT JOIN fund AS f ON i.fund_id=f.id
INNER JOIN (SELECT *
           FROM funding_round
           WHERE EXTRACT (YEAR FROM funded_at) BETWEEN 2012 AND 2013) as fr ON fr.id=i.funding_round_id
WHERE c.milestones > 6
'''
pd.io.sql.read_sql(query, con = engine)

20.Выгрузите таблицу, в которой будут такие поля:
* название компании-покупателя;
* сумма сделки;
* название компании, которую купили;
* сумма инвестиций, вложенных в купленную компанию;
* доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
query = '''
SELECT c.name AS acquiring_company,
       a.price_amount,
       c1.name AS acquired_company,
       c1.funding_total,
       ROUND(a.price_amount/c1.funding_total)
FROM acquisition AS a
LEFT JOIN company AS c ON a.acquiring_company_id=c.id
LEFT JOIN company AS c1 ON a.acquired_company_id=c1.id
WHERE a.price_amount <> 0
AND c1.funding_total>0
ORDER BY a.price_amount DESC
LIMIT 10;
'''
pd.io.sql.read_sql(query, con = engine)

21.Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
query = '''
SELECT name,
       EXTRACT(MONTH FROM CAST(funded_at AS timestamp)) AS month
FROM company AS c LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE category_code = 'social'
AND EXTRACT (YEAR FROM CAST(funded_at AS timestamp)) BETWEEN 2010 AND 2013
'''
pd.io.sql.read_sql(query, con = engine)

22.Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
* номер месяца, в котором проходили раунды;
* количество уникальных названий фондов из США, которые инвестировали в этом месяце;
* количество компаний, купленных за этот месяц;
* общая сумма сделок по покупкам в этом месяце.

In [ ]:
query = '''
WITH
t1 AS (SELECT EXTRACT(MONTH FROM CAST(funded_at AS timestamp)) AS round_month,
       COUNT(DISTINCT(f.name)) AS fund_company
FROM funding_round as fr LEFT JOIN  investment AS i ON fr.id = i.funding_round_id
LEFT JOIN fund as f ON i.fund_id = f.id
WHERE EXTRACT(YEAR FROM CAST(funded_at AS timestamp)) BETWEEN 2010 AND 2013
AND country_code = 'USA'
GROUP BY round_month),

t2 AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS timestamp)) AS acq_month,
       COUNT(acquired_company_id) AS acq_company,
       SUM(price_amount) AS sum_price
FROM acquisition
WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
GROUP BY acq_month)

SELECT round_month,
       fund_company,
       acq_company,
       sum_price
FROM t1 JOIN t2 ON round_month = acq_month
'''
pd.io.sql.read_sql(query, con = engine)

23.Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
query = '''
WITH
y2011 AS (SELECT country_code AS cc11,
       AVG(funding_total) AS avg_2011
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = '2011'
GROUP BY country_code),

y2012 AS (SELECT country_code AS cc12,
       AVG(funding_total) AS avg_2012
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = '2012'
GROUP BY country_code),

y2013 AS (SELECT country_code AS cc13,
       AVG(funding_total) AS avg_2013
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = '2013'
GROUP BY country_code)

SELECT cc11,
       avg_2011,
       avg_2012,
       avg_2013
FROM y2011 JOIN y2012 ON cc11=cc12 
JOIN y2013 ON cc11=cc13
ORDER BY avg_2011 DESC
'''
pd.io.sql.read_sql(query, con = engine)